In [1]:
# Dependencies
import json
import os
import pandas as pd

In [2]:
#open json
object_path = 'sample_data.json'
with open(object_path) as jsonfile:
       data_json = json.load(jsonfile)
df_from_json = pd.DataFrame(data_json)

In [3]:
df_from_json

,NCTId,ArmGroupInterventionName,ArmGroupType,BaselineMeasureDispersionType,BaselineMeasureParamType,CollaboratorClass,ConditionAncestorId,ConditionBrowseBranchAbbrev,ConditionBrowseLeafId,ConditionMeshId,...,InterventionType,IsFDARegulatedDevice,IsFDARegulatedDrug,LeadSponsorClass,OverallStatus,PatientRegistry,Phase,SamplingMethod,StdAge,StudyType
0,NCT04371159,Device: Velieve U.S.,Experimental,0,0,0,"[D000007239, D000014570]","[BC01, All, BXS]","[M8866, M4951, M15885, M15902]",D000014552,...,Device,Yes,No,INDUSTRY,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
1,NCT04474496,Other: Assessing impact of COVID19,0,0,0,OTHER,0,"[BC18, BC19, All]","[M5698, M5702]",0,...,Other,No,No,OTHER,Enrolling by invitation,No,0,Non-Probability Sample,"[Adult, Older Adult]",Observational
2,NCT02801266,Behavioral: Evidence informed birth control co...,"[Experimental, No Intervention]",0,0,OTHER,0,0,0,0,...,Behavioral,0,0,OTHER,Completed,0,Not Applicable,0,Adult,Interventional
3,NCT01599390,Other: Data collection,0,0,0,INDUSTRY,"[D000009976, D000012327, D000014777, D00001214...","[BC02, BC08, All, BC01]","[M8878, M8866, M4951, M11485, M16105, M13732, ...",D000007251,...,Other,0,0,INDUSTRY,Completed,0,0,Non-Probability Sample,"[Child, Adult, Older Adult]",Observational
4,NCT00312702,[Biological: Falciparum Malaria Protein 11 wit...,"[Experimental, Experimental]",Standard Deviation,"[Mean, Count of Participants, Count of Partici...","[INDUSTRY, OTHER, FED]","[D000011528, D000010272]","[BC03, All, BC01, Rare]","[M17716, M9863, M8866, M4951, M12971, M11768, ...","[D000008288, D000016778]",...,Biological,0,0,FED,Completed,0,"[Phase 1, Phase 2]",0,Adult,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NCT03768310,"[Biological: CD19.CAR-multiVST for Group A, Bi...","[Experimental, Experimental]",0,0,"[OTHER, OTHER]","[D000009370, D000009369, D000008232, D00000820...","[BC04, All, BC15, BC20, Rare]","[M9528, M9803, M26172, M9534, M10898, M9808, M...",D000054198,...,"[Biological, Biological]",No,Yes,OTHER,Not yet recruiting,0,Phase 1,0,"[Child, Adult, Older Adult]",Interventional
9996,NCT03768206,"[Behavioral: Standard Physical Therapy, Behavi...","[Experimental, Active Comparator]",0,0,OTHER,0,0,0,0,...,"[Behavioral, Behavioral, Behavioral]",No,No,OTHER,Completed,0,Not Applicable,0,"[Adult, Older Adult]",Interventional
9997,NCT03767738,Drug: Intravitreal Aflibercept Injection (IAI),Experimental,0,0,0,D000002318,"[BC14, All]",M15983,D000014652,...,Drug,No,Yes,INDUSTRY,Completed,0,Phase 4,0,"[Adult, Older Adult]",Interventional
9998,NCT03767595,Device: ProACT Adjustable Continence Therapy f...,Experimental,0,0,0,"[D000014555, D000014570, D000059411, D000020924]","[BXS, BC23, All, BXM]","[M15882, M6519, M15883, M15888, M15902, M28051...","[D000014549, D000014550]",...,Device,Yes,No,INDUSTRY,Recruiting,0,Not Applicable,0,"[Adult, Older Adult]",Interventional


In [4]:
# explodeTable function - to open a column from the list and create a new table from it
def explodeTable(df_name, column_name):
    tb = df_name[['NCTId', column_name]].copy()
    explode_tb = tb.explode(column_name)
    explode_tb = explode_tb.replace(0, "-")
    explode_tb = explode_tb.sort_values(by=[column_name], ascending=False)
    fr = len(explode_tb) - explode_tb[column_name].str.contains('-', na=False).sum()
    print('feature_row', fr)
    feature_tb = explode_tb.head(fr)
    #.rename(columns={column_name:'features'})

    return feature_tb

In [5]:
# extract "OverallStatus" value 
status_tb = df_from_json[['NCTId', 'OverallStatus']].copy()
status_tb = status_tb.explode('OverallStatus')
status_tb.sort_values('NCTId', inplace=True)
status_tb.reset_index(drop=True, inplace=True)
status_tb

,NCTId,OverallStatus
0,NCT00000152,Unknown status
1,NCT00000157,Terminated
2,NCT00000172,Completed
3,NCT00000173,Completed
4,NCT00000174,Completed
...,...,...
9995,NCT04608188,Recruiting
9996,NCT04608773,Not yet recruiting
9997,NCT04609085,Enrolling by invitation
9998,NCT04609514,Not yet recruiting


In [6]:
# extract "Phase" value
phase_tb = df_from_json[['NCTId', 'Phase']].copy()
phase_tb = phase_tb.explode('Phase')
phase_tb = phase_tb.replace({'Not Applicable':'Phase_n/a',0:'Phase_n/a','Early Phase 1': 'Phase_1', 'Phase 1': 'Phase_1', 'Phase 2': 'Phase_2', 'Phase 3': 'Phase_3', 'Phase 4': 'Phase_4'})
phase_tb

,NCTId,Phase
0,NCT04371159,Phase_n/a
1,NCT04474496,Phase_n/a
2,NCT02801266,Phase_n/a
3,NCT01599390,Phase_n/a
4,NCT00312702,Phase_1
...,...,...
9995,NCT03768310,Phase_1
9996,NCT03768206,Phase_n/a
9997,NCT03767738,Phase_4
9998,NCT03767595,Phase_n/a


In [7]:
# create table to split the Phases
phase_crosstb = pd.crosstab(index=phase_tb['NCTId'], columns = phase_tb['Phase']).reset_index()
phase_crosstb.sort_values('NCTId', inplace=True)
phase_crosstb

Phase,NCTId,Phase_1,Phase_2,Phase_3,Phase_4,Phase_n/a
0,NCT00000152,0,0,1,0,0
1,NCT00000157,0,0,1,0,0
2,NCT00000172,0,0,1,0,0
3,NCT00000173,0,0,1,0,0
4,NCT00000174,0,0,1,0,0
...,...,...,...,...,...,...
9995,NCT04608188,0,0,0,0,1
9996,NCT04608773,0,0,0,0,1
9997,NCT04609085,0,0,0,0,1
9998,NCT04609514,0,0,0,0,1


# create a new table with "Status"
df = df_from_json.copy()
df= df.drop(columns=['OverallStatus', 'Phase'])
df.insert(1,'Status',status_tb['OverallStatus'])
df.sort_values('NCTId', inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [8]:
# attach the Phases to the new table
df = pd.concat([status_tb,phase_crosstb], axis=1, sort=False)
df

,NCTId,OverallStatus,NCTId,Phase_1,Phase_2,Phase_3,Phase_4,Phase_n/a
0,NCT00000152,Unknown status,NCT00000152,0,0,1,0,0
1,NCT00000157,Terminated,NCT00000157,0,0,1,0,0
2,NCT00000172,Completed,NCT00000172,0,0,1,0,0
3,NCT00000173,Completed,NCT00000173,0,0,1,0,0
4,NCT00000174,Completed,NCT00000174,0,0,1,0,0
...,...,...,...,...,...,...,...,...
9995,NCT04608188,Recruiting,NCT04608188,0,0,0,0,1
9996,NCT04608773,Not yet recruiting,NCT04608773,0,0,0,0,1
9997,NCT04609085,Enrolling by invitation,NCT04609085,0,0,0,0,1
9998,NCT04609514,Not yet recruiting,NCT04609514,0,0,0,0,1


In [11]:
df = df.drop(df.columns[2], axis=1)
df

,OverallStatus,Phase_1,Phase_2,Phase_3,Phase_4,Phase_n/a
0,Unknown status,0,0,1,0,0
1,Terminated,0,0,1,0,0
2,Completed,0,0,1,0,0
3,Completed,0,0,1,0,0
4,Completed,0,0,1,0,0
...,...,...,...,...,...,...
9995,Recruiting,0,0,0,0,1
9996,Not yet recruiting,0,0,0,0,1
9997,Enrolling by invitation,0,0,0,0,1
9998,Not yet recruiting,0,0,0,0,1
